In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import json
import h3
import folium
import osmnx as ox
from shapely import wkt
from folium.plugins import HeatMap
from shapely.geometry import Polygon

In [2]:
def visualize_hexagons(hexagons, color="red", folium_map=None):

    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)

    if folium_map is None:
        m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=20, tiles='cartodbpositron')
    else:
        m = folium_map

    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color=color)
        m.add_child(my_PolyLine)
    return m

h3_address = h3.geo_to_h3(45.035470, 38.975313,  9) # 9 - индекс, определяющий размер гексагона
visualize_hexagons([h3_address])

In [3]:
def visualize_polygons(geometry):

    lats, lons = get_lat_lon(geometry)

    m = folium.Map(location=[sum(lats)/len(lats), sum(lons)/len(lons)], zoom_start=13, tiles='cartodbpositron')

    overlay = gpd.GeoSeries(geometry).to_json()
    folium.GeoJson(overlay, name = 'boundary').add_to(m)

    return m

# выводим центроиды полигонов
def get_lat_lon(geometry):

    lon = geometry.apply(lambda x: x.x if x.type == 'Point' else x.centroid.x)
    lat = geometry.apply(lambda x: x.y if x.type == 'Point' else x.centroid.y)
    return lat, lon

# выгрузим границы Краснодара из OSM
cities = ['Архангельск']
polygon_arh = ox.geometries_from_place(cities, {'boundary':'administrative'}).reset_index()
polygon_arh = polygon_arh[(polygon_arh['name'] == 'городской округ Архангельск')]
# посмотрим что получилось
visualize_polygons(polygon_arh['geometry'])

C:\Users\igorb\miniconda3\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
C:\Users\igorb\miniconda3\lib\site-packages\osmnx\geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [4]:
def create_hexagons(geoJson):

    polyline = geoJson['coordinates'][0]

    polyline.append(polyline[0])
    lat = [p[0] for p in polyline]
    lng = [p[1] for p in polyline]
    m = folium.Map(location=[sum(lat)/len(lat), sum(lng)/len(lng)], zoom_start=13, tiles='cartodbpositron')
    my_PolyLine=folium.PolyLine(locations=polyline,weight=8,color="green")
    m.add_child(my_PolyLine)

    hexagons = list(h3.polyfill(geoJson, 8))
    polylines = []
    lat = []
    lng = []
    for hex in hexagons:
        polygons = h3.h3_set_to_multi_polygon([hex], geo_json=False)
        # flatten polygons into loops.
        outlines = [loop for polygon in polygons for loop in polygon]
        polyline = [outline + [outline[0]] for outline in outlines][0]
        lat.extend(map(lambda v:v[0],polyline))
        lng.extend(map(lambda v:v[1],polyline))
        polylines.append(polyline)
    for polyline in polylines:
        my_PolyLine=folium.PolyLine(locations=polyline,weight=3,color='red')
        m.add_child(my_PolyLine)

    polylines_x = []
    for j in range(len(polylines)):
        a = np.column_stack((np.array(polylines[j])[:,1],np.array(polylines[j])[:,0])).tolist()
        polylines_x.append([(a[i][0], a[i][1]) for i in range(len(a))])

    polygons_hex = pd.Series(polylines_x).apply(lambda x: Polygon(x))

    return m, polygons_hex, polylines
# polygon_hex , polylines - геометрии гексагонов в разных форматах

# сгенерим гексагоны внутри полигона г. Краснодар
geoJson = json.loads(gpd.GeoSeries(polygon_arh['geometry']).to_json())
geoJson = geoJson['features'][0]['geometry']
geoJson = {'type':'Polygon','coordinates': [np.column_stack((np.array(geoJson['coordinates'][0])[:, 1], np.array(geoJson['coordinates'][0])[:, 0])).tolist()]}

m, polygons, polylines = create_hexagons(geoJson)
m

In [5]:
def osm_query(tag, city):
    gdf = ox.geometries_from_place(city, tag).reset_index()
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    print(gdf.shape)
    return gdf

 # Выгрузим интересующие нас категории объектов
tags = [
        {'building' : 'apartments'}, {'building' : 'detached'},
        {'building' : 'dormitory'}, {'building' : 'hotel'},
        {'building' : 'house'}, {'building' : 'semidetached_house'},
        {'building' : 'terrace'},  {'building' : 'commercial'},
        {'building' : 'office'},  {'building' : 'terrace'},
        {'building' : 'terrace'}, {'building':'retail'},
        {'building':'train_station'}, {'building':'flats'},
        {'building':'levels'},
        {'highway' : 'bus_stop'}, {'footway':'crossing'},
        {'amenity':'cafe'}, {'amenity':'fast_food'},
        {'amenity':'restaurant'}, {'amenity':'college'},
        {'amenity':'language_school'},  {'amenity':'school'},
        {'amenity':'university'},  {'amenity':'atm'},
        {'amenity':'bank'},  {'amenity':'clinic'},
        {'amenity':'hospital'},  {'amenity':'pharmacy'},
        {'amenity':'theatre'},  {'amenity':'townhall'},
        {'amenity':'bench'}, {'amenity':'courthouse'},
        {'amenity': 'parking'},
        {'aeroway': 'aerodrome'}
       ]
cities = ['Архангельск, Россия']

gdfs = []
for city in cities:
    for tag in tags:
        gdfs.append(osm_query(tag, city))

# посмотрим что получилось
data_poi = pd.concat(gdfs)
data_poi.groupby(['city','object','type'], as_index = False).agg({'geometry':'count'})

# добавим координаты/центроиды
lat, lon = get_lat_lon(data_poi['geometry'])
data_poi['lat'] = lat
data_poi['lon'] = lon

gdf_1 = gpd.GeoDataFrame(data_poi, geometry=gpd.points_from_xy(data_poi.lon, data_poi.lat))

gdf_2 = pd.DataFrame(polygons, columns = ['geometry'])
gdf_2['polylines'] = polylines
gdf_2['geometry'] = gdf_2['geometry'].astype(str)
geometry_uniq = pd.DataFrame(gdf_2['geometry'].drop_duplicates())
geometry_uniq['id'] = np.arange(len(geometry_uniq)).astype(str)
gdf_2 = gdf_2.merge(geometry_uniq, on = 'geometry')
gdf_2['geometry'] = gdf_2['geometry'].apply(wkt.loads)
gdf_2 = gpd.GeoDataFrame(gdf_2, geometry='geometry')

itog_table = gpd.sjoin(gdf_2, gdf_1, how='left', op='intersects')
itog_table = itog_table.dropna()
itog_table.head()

(3319, 4)
(0, 4)
(25, 4)
(4, 4)
(1500, 4)
(0, 4)
(0, 4)
(209, 4)
(42, 4)
(0, 4)
(0, 4)
(381, 4)
(3, 4)
(0, 4)
(0, 4)
(407, 4)
(584, 4)
(238, 4)
(149, 4)
(34, 4)
(26, 4)
(0, 4)
(102, 4)
(33, 4)
(307, 4)
(84, 4)
(43, 4)
(41, 4)
(239, 4)
(7, 4)
(11, 4)
(783, 4)
(14, 4)
(1344, 4)
(1, 4)


,geometry,polylines,id,index_right,city,object,type,lat,lon
0,"POLYGON ((40.51522 64.66403, 40.52064 64.66031...","[(64.66402971369125, 40.51522301647388), (64.6...",0,323.0,Архангельск,highway,bus_stop,64.665410,40.526785
0,"POLYGON ((40.51522 64.66403, 40.52064 64.66031...","[(64.66402971369125, 40.51522301647388), (64.6...",0,324.0,Архангельск,highway,bus_stop,64.666088,40.525742
1,"POLYGON ((40.59016 64.52775, 40.58477 64.53149...","[(64.52775242000696, 40.59016013648966), (64.5...",1,138.0,Архангельск,building,retail,64.526914,40.581875
1,"POLYGON ((40.59016 64.52775, 40.58477 64.53149...","[(64.52775242000696, 40.59016013648966), (64.5...",1,181.0,Архангельск,building,apartments,64.527056,40.588676
1,"POLYGON ((40.59016 64.52775, 40.58477 64.53149...","[(64.52775242000696, 40.59016013648966), (64.5...",1,106.0,Архангельск,highway,bus_stop,64.527201,40.582567


In [6]:
def create_choropleth(data, json, columns, legend_name, feature, bins):

    lat, lon = get_lat_lon(data['geometry'])

    m = folium.Map(location=[sum(lat)/len(lat), sum(lon)/len(lon)], zoom_start=13, tiles='cartodbpositron')

    folium.Choropleth(
        geo_data=json,
        name="choropleth",
        data=data,
        columns=columns,
        key_on="feature.id",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name=legend_name,
        nan_fill_color = 'black',
        bins = bins

    ).add_to(m)

    folium.LayerControl().add_to(m)

    return m

# подготовим данные
itog_table['geometry'] = itog_table['geometry'].astype(str) #для groupby
itog_table['id'] = itog_table['id'].astype(str) #для Choropleth
agg_all = itog_table.groupby(['geometry','type','id'], as_index = False).agg({'lat':'count'}).rename(columns = {'lat':'counts'})
agg_all['geometry'] = agg_all['geometry'].apply(wkt.loads) #возвращаем формат геометрий

agg_all_cafe = agg_all.query("type == 'pharmacy'")[["geometry","counts",'id']]
agg_all_cafe['id'] = agg_all_cafe['id'].astype(str)
data_geo_1 = gpd.GeoSeries(agg_all_cafe.set_index('id')["geometry"]).to_json()

create_choropleth(agg_all_cafe, data_geo_1, ["id","counts"], 'courthouse counts', 'counts', 5)

C:\Users\igorb\miniconda3\lib\site-packages\geopandas\geodataframe.py:1321: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")
